### Data wrangling 1

In this part, we preprocess data for EDA and predictive modeling.

Specifically, we do the following:
1. Check and clean duplicate reviews.
2. Remove barbershops since we are interested in restaurants in this project.
3. Remove closed restaurants.
4. Convert user_id and business_id to integers (to save space and speed up computation).

In [1]:
import pandas as pd
import numpy as np

from IPython.display import display

import matplotlib
import matplotlib.pyplot as plt

import seaborn as sns
pd.set_option('display.width', 15000)
pd.set_option('display.max_columns', 100)
sns.set_style("whitegrid", {'axes.grid' : False})
sns.set_context('poster')
%matplotlib inline

In [2]:
class id2num:
    def __init__(self):
        pass
    
    def _id2num_dict(self, df, col):
        _num2id = df[col].values
        _id2num = {v:n for n, v in enumerate(_num2id)}
        return _num2id, _id2num
    
    def _id2num(self, df, col, i2n):
        def transform(v):
            return i2n[v]
        df[col] = df[col].apply(transform)
        return df
    
    def fit_transform(self, dfr, dfb, dfu):
        bus_n2i, bus_i2n = self._id2num_dict(dfb, 'business_id') 
        user_n2i, user_i2n = self._id2num_dict(dfu, 'user_id')
        dfb = self._id2num(dfb, 'business_id', bus_i2n)
        dfu = self._id2num(dfu, 'user_id', user_i2n)
        dfr = self._id2num(dfr, 'business_id', bus_i2n)
        dfr = self._id2num(dfr, 'user_id', user_i2n)
        return dfr, dfb, dfu

In [3]:
%%time
# Load data and convert user_id and business_id to integers

raw_dir = 'raw/'
df_business = pd.read_csv(raw_dir + 'business.csv')
df_review = pd.read_csv(raw_dir + 'review.csv')
df_user = pd.read_csv(raw_dir + 'user.csv')

df_review, df_business, df_user = id2num().fit_transform(df_review, df_business, df_user)
data_review = df_review[['user_id', 'business_id', 'stars']]

Wall time: 1min 5s


In [4]:
print('The number of missing values in review.stars is {}.'.format(np.sum(df_review.stars.isnull())))

The number of missing values in review.stars is 0.


In [5]:
# check duplicates

df_review.groupby(['user_id', 'business_id'], as_index=False).count()\
[df_review.groupby(['user_id', 'business_id'], as_index=False).count() != 1].dropna()[['user_id', 'business_id', 'stars']]

,user_id,business_id,stars
1773114,98431.0,63954.0,2.0


In [6]:
df_review.query('user_id == 98431 & business_id == 63954')

,funny,user_id,review_id,text,business_id,stars,date,useful,cool
4291340,0,98431,1l-mU-BwQsZRW7BKdHVwZw,"This place continues to be a shit hole, with P...",63954,1,2017-05-02,0,0
4291341,0,98431,IcpPtd37iJEthZybuasS4g,My mother recently spent a good deal of time i...,63954,1,2015-02-06,9,0


As we can see, there is only one case where a user rate the same restaurant more than once and the ratings are the same in this case. We can simply drop the duplicate.

In [7]:
df_review = df_review.drop(4291341)
df_review.query('user_id == 98431 & business_id == 63954')

,funny,user_id,review_id,text,business_id,stars,date,useful,cool
4291340,0,98431,1l-mU-BwQsZRW7BKdHVwZw,"This place continues to be a shit hole, with P...",63954,1,2017-05-02,0,0


In [8]:
def remove_barbershop(df_business=df_business, df_review=df_review, df_user=df_user):
    b_cols = [col for col in df_business.columns if 'HairSpecializesIn' in col]
    df_business = df_business[np.all(df_business[b_cols].isnull(), axis=1)].drop(156300)
    for col in b_cols:
        del df_business[col]
    df_review = pd.merge(df_business[['business_id']], df_review, how='left', on='business_id')
    data_review = df_review[['user_id', 'business_id', 'stars']]
    df_user = pd.merge(pd.DataFrame(data_review['user_id'].unique(), \
                                    columns=['user_id']), df_user, how='left', on='user_id')
    return df_business, df_review, df_user, data_review

In [9]:
%%time
df_business, df_review, df_user, data_review = remove_barbershop()

Wall time: 9.12 s


In [10]:
def remove_closed(df_business=df_business, df_review=df_review, df_user=df_user):
    df_business = df_business[df_business['is_open'] == 1]
    df_review = pd.merge(df_business[['business_id']], df_review, how='left', on='business_id')
    df_user = pd.merge(pd.DataFrame(df_review['user_id'].unique(), \
                                    columns=['user_id']), df_user, how='left', on='user_id')
    df_review, df_business, df_user = id2num().fit_transform(df_review, df_business, df_user)
    data_review = df_review[['user_id', 'business_id', 'stars']]
    return df_business, df_review, df_user, data_review

In [11]:
%%time
df_business, df_review, df_user, data_review = remove_closed()

C:\Users\Weidong\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]


Wall time: 13.7 s


In [12]:
%%time
# Save data
data_dir = 'data/Full/'
df_review.to_pickle(data_dir + 'review.pkl')
df_business.to_pickle(data_dir + 'business.pkl')
df_user.to_pickle(data_dir + 'user.pkl')
data_review.to_pickle(data_dir + 'data_review.pkl')

Wall time: 34.2 s


In [13]:
df_business[['city', 'name']].groupby('city', as_index=False).count().sort_values(by='name', ascending=False)

,city,name
381,Las Vegas,20434
632,Phoenix,13152
880,Toronto,12118
128,Charlotte,6347
780,Scottsdale,6037
651,Pittsburgh,4745
473,Mesa,4470
508,Montréal,4308
303,Henderson,3465
872,Tempe,3156


In [14]:
def get_city(city, df_business=df_business, df_review=df_review, df_user=df_user):
    dfb = df_business[df_business['city'] == city]
    dfr = pd.merge(dfb[['business_id']], df_review, how='left', on='business_id')
    datar = dfr[['user_id', 'business_id', 'stars']]
    dfu = pd.merge(pd.DataFrame(datar['user_id'].unique(), columns=['user_id']), df_user, how='left', on='user_id')
    return dfb, dfr, dfu, datar

In [15]:
%%time
dfb, dfr, dfu, datar = get_city('Las Vegas')
print(dfr.shape)
data_dir = 'data/Las_Vegas/'
dfr.to_pickle(data_dir + 'review.pkl')
dfb.to_pickle(data_dir + 'business.pkl')
dfu.to_pickle(data_dir + 'user.pkl')
datar.to_pickle(data_dir + 'data_review.pkl')

(1280896, 9)
Wall time: 15.7 s


In [16]:
%%time
dfb, dfr, dfu, datar = get_city('Champaign')
print(dfr.shape)
data_dir = 'data/Champaign/'
dfr.to_pickle(data_dir + 'review.pkl')
dfb.to_pickle(data_dir + 'business.pkl')
dfu.to_pickle(data_dir + 'user.pkl')
datar.to_pickle(data_dir + 'data_review.pkl')

(20571, 9)
Wall time: 1.47 s
